## 名前付きエンティティ認識（NER）によるテキストの拡張

`Named Entity Recognition`（NER）は、`Natural Language Processing`タスクの一つで、固有表現（NE）を識別し、事前に定義された意味カテゴリ（人物、組織、場所、イベント、時間表現、数量など）に分類します。生のテキストを構造化された情報に変換することで、NERはデータをより実用的にし、情報抽出、データ集約、分析、ソーシャルメディア監視などのタスクを促進します。

このノートブックでは、[chat completion](https://platform.openai.com/docs/api-reference/chat)と[functions-calling](https://platform.openai.com/docs/guides/gpt/function-calling)を使用してNERを実行し、Wikipediaなどの知識ベースへのリンクでテキストを豊かにする方法を示します：

**テキスト:**

*In Germany, in 1440, goldsmith Johannes Gutenberg invented the movable-type printing press. His work led to an information revolution and the unprecedented mass-spread of literature throughout Europe. Modelled on the design of the existing screw presses, a single Renaissance movable-type printing press could produce up to 3,600 pages per workday.*

**Wikipediaリンクで豊かにされたテキスト:**

*In [Germany](https://en.wikipedia.org/wiki/Germany), in 1440, goldsmith [Johannes Gutenberg]() invented the [movable-type printing press](https://en.wikipedia.org/wiki/Movable_Type). His work led to an [information revolution](https://en.wikipedia.org/wiki/Information_revolution) and the unprecedented mass-spread of literature throughout [Europe](https://en.wikipedia.org/wiki/Europe). Modelled on the design of the existing screw presses, a single [Renaissance](https://en.wikipedia.org/wiki/Renaissance) [movable-type printing press](https://en.wikipedia.org/wiki/Movable_Type) could produce up to 3,600 pages per workday.*

**推論コスト:** このノートブックでは、OpenAI APIのコストを見積もる方法も説明します。

### 1. セットアップ

#### 1.1 Pythonパッケージのインストール/アップグレード

In [1]:
%pip install --upgrade openai --quiet
%pip install --upgrade nlpia2-wikipedia --quiet
%pip install --upgrade tenacity --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


#### 1.2 パッケージとOPENAI_API_KEYの読み込み

OpenAI のウェブインターフェースでAPIキーを生成できます。詳細については https://platform.openai.com/account/api-keys を参照してください。

このノートブックは、最新のOpenAIモデル`gpt-3.5-turbo-0613`と`gpt-4-0613`で動作します。

In [2]:
import json
import logging
import os

import openai
import wikipedia

from typing import Optional
from IPython.display import display, Markdown
from tenacity import retry, wait_random_exponential, stop_after_attempt

logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

OPENAI_MODEL = 'gpt-3.5-turbo-0613'

client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

### 2. 識別するNERラベルの定義

幅広いユースケースを示すために、標準的なNERラベルセットを定義します。ただし、テキストをナレッジベースのリンクで豊富化するという我々の特定のタスクにおいては、実際にはそのサブセットのみが必要です。

In [3]:
labels = [
    "person",      # people, including fictional characters
    "fac",         # buildings, airports, highways, bridges
    "org",         # organizations, companies, agencies, institutions
    "gpe",         # geopolitical entities like countries, cities, states
    "loc",         # non-gpe locations
    "product",     # vehicles, foods, appareal, appliances, software, toys 
    "event",       # named sports, scientific milestones, historical events
    "work_of_art", # titles of books, songs, movies
    "law",         # named laws, acts, or legislations
    "language",    # any named language
    "date",        # absolute or relative dates or periods
    "time",        # time units smaller than a day
    "percent",     # percentage (e.g., "twenty percent", "18%")
    "money",       # monetary values, including unit
    "quantity",    # measurements, e.g., weight or distance
]

### 3. メッセージを準備する

[chat completions API](https://platform.openai.com/docs/guides/gpt/chat-completions-api)は、メッセージのリストを入力として受け取り、モデルが生成したメッセージを出力として提供します。チャット形式は主に複数ターンの会話を促進するために設計されていますが、事前の会話がない単一ターンのタスクにも同様に効率的です。今回の目的では、system、assistant、userの各役割に対してメッセージを指定します。

#### 3.1 システムメッセージ

`system message`（プロンプト）は、アシスタントの望ましいペルソナとタスクを定義することで、アシスタントの動作を設定します。また、識別を目指す特定のエンティティラベルのセットも明確に定義します。

モデルにレスポンスの形式を指示することは可能ですが、`gpt-3.5-turbo-0613`と`gpt-4-0613`の両方が、関数をいつ呼び出すべきかを判断し、関数のシグネチャに従ってフォーマットされた`JSON`で応答するようにファインチューニングされていることに注意する必要があります。この機能により、プロンプトが合理化され、モデルから直接構造化されたデータを受け取ることができます。

In [4]:
def system_message(labels):
    return f"""
You are an expert in Natural Language Processing. Your task is to identify common Named Entities (NER) in a given text.
The possible common Named Entities (NER) types are exclusively: ({", ".join(labels)})."""

申し訳ございませんが、翻訳すべきテキストが提供されていないようです。「#### 3.2 Assistant Message」という見出しのみが表示されています。

翻訳したい英語のテキスト全体を提供していただけますでしょうか？そうすれば、上記のルールに従って適切に日本語に翻訳いたします。

`Assistant messages`は通常、以前のアシスタントの応答を保存します。しかし、私たちのシナリオのように、望ましい動作の例を提供するために作成することもできます。OpenAIは`zero-shot`の固有表現認識を実行できますが、`one-shot`アプローチの方がより正確な結果を生成することがわかっています。

In [5]:
def assisstant_message():
    return f"""
EXAMPLE:
    Text: 'In Germany, in 1440, goldsmith Johannes Gutenberg invented the movable-type printing press. His work led to an information revolution and the unprecedented mass-spread / 
    of literature throughout Europe. Modelled on the design of the existing screw presses, a single Renaissance movable-type printing press could produce up to 3,600 pages per workday.'
    {{
        "gpe": ["Germany", "Europe"],
        "date": ["1440"],
        "person": ["Johannes Gutenberg"],
        "product": ["movable-type printing press"],
        "event": ["Renaissance"],
        "quantity": ["3,600 pages"],
        "time": ["workday"]
    }}
--"""

#### 3.3 ユーザーメッセージ

`user message`は、アシスタントタスクの具体的なテキストを提供します：

In [6]:
def user_message(text):
    return f"""
TASK:
    Text: {text}
"""

### 4. OpenAI Functions（およびユーティリティ）

OpenAI APIコールにおいて、`gpt-3.5-turbo-0613`と`gpt-4-0613`に`functions`を記述し、モデルがそれらの`functions`を呼び出すための引数を含むJSONオブジェクトを出力するよう知的に選択させることができます。重要な点として、[chat completions API](https://platform.openai.com/docs/guides/gpt/chat-completions-api)は実際には`function`を実行しないということです。代わりに、JSONの出力を提供し、それを使用してコード内で`function`を呼び出すことができます。詳細については、[OpenAI Function Calling Guide](https://platform.openai.com/docs/guides/function-calling)を参照してください。

私たちの関数 `enrich_entities(text, label_entities)` は、テキストのブロックと、識別されたラベルとエンティティを含む辞書をパラメータとして受け取ります。そして、認識されたエンティティを対応するWikipediaの記事へのリンクと関連付けます。

In [7]:
@retry(wait=wait_random_exponential(min=1, max=10), stop=stop_after_attempt(5))
def find_link(entity: str) -> Optional[str]:
    """
    Finds a Wikipedia link for a given entity.
    """
    try:
        titles = wikipedia.search(entity)
        if titles:
            # naively consider the first result as the best
            page = wikipedia.page(titles[0])
            return page.url
    except (wikipedia.exceptions.WikipediaException) as ex:
        logging.error(f'Error occurred while searching for Wikipedia link for entity {entity}: {str(ex)}')

    return None

In [8]:
def find_all_links(label_entities:dict) -> dict:
    """ 
    Finds all Wikipedia links for the dictionary entities in the whitelist label list.
    """
    whitelist = ['event', 'gpe', 'org', 'person', 'product', 'work_of_art']
    
    return {e: find_link(e) for label, entities in label_entities.items() 
                            for e in entities
                            if label in whitelist}

In [9]:
def enrich_entities(text: str, label_entities: dict) -> str:
    """
    Enriches text with knowledge base links.
    """
    entity_link_dict = find_all_links(label_entities)
    logging.info(f"entity_link_dict: {entity_link_dict}")
    
    for entity, link in entity_link_dict.items():
        text = text.replace(entity, f"[{entity}]({link})")

    return text

### 4. ChatCompletion

前述のとおり、`gpt-3.5-turbo-0613`と`gpt-4-0613`は、`function`を呼び出すべきタイミングを検出するように微調整されています。さらに、これらのモデルは`function`のシグネチャに準拠した`JSON`レスポンスを生成することができます。以下の手順に従います：

1. `function`とそれに関連する`JSON`スキーマを定義する
2. `messages`、`tools`、`tool_choice`パラメータを使用してモデルを呼び出す
3. 出力を`JSON`オブジェクトに変換し、モデルが提供した`arguments`を使って`function`を呼び出す

実際の運用では、`function`のレスポンスを新しいメッセージとして追加してモデルを再度呼び出し、結果をユーザーに要約させることが望ましい場合があります。ただし、今回の目的においては、このステップは必要ありません。

*実際のユースケースでは、アクションを実行する前にユーザー確認フローを組み込むことを強く推奨します。*

#### 4.1 関数とJSONスキーマの定義

モデルにラベルと認識されたエンティティの辞書を出力させたいので：

```python
{   
    "gpe": ["Germany", "Europe"],   
    "date": ["1440"],   
    "person": ["Johannes Gutenberg"],   
    "product": ["movable-type printing press"],   
    "event": ["Renaissance"],   
    "quantity": ["3,600 pages"],   
    "time": ["workday"]   
}   
```

`tools`パラメータに渡すための対応するJSONスキーマを定義する必要があります：

In [10]:
def generate_functions(labels: dict) -> list:
    return [
        {   
            "type": "function",
            "function": {
                "name": "enrich_entities",
                "description": "Enrich Text with Knowledge Base Links",
                "parameters": {
                    "type": "object",
                        "properties": {
                            "r'^(?:' + '|'.join({labels}) + ')$'": 
                            {
                                "type": "array",
                                "items": {
                                    "type": "string"
                                }
                            }
                        },
                        "additionalProperties": False
                },
            }
        }
    ]

#### 4.2 チャット補完

次に、モデルを呼び出します。`tool_choice`パラメータを`{"type": "function", "function" : {"name": "enrich_entities"}}`に設定することで、APIに特定の関数を使用するよう指示していることに注意することが重要です。

In [11]:
@retry(wait=wait_random_exponential(min=1, max=10), stop=stop_after_attempt(5))
def run_openai_task(labels, text):
    messages = [
          {"role": "system", "content": system_message(labels=labels)},
          {"role": "assistant", "content": assisstant_message()},
          {"role": "user", "content": user_message(text=text)}
      ]

    # TODO: functions and function_call are deprecated, need to be updated
    # See: https://platform.openai.com/docs/api-reference/chat/create#chat-create-tools
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        tools=generate_functions(labels),
        tool_choice={"type": "function", "function" : {"name": "enrich_entities"}}, 
        temperature=0,
        frequency_penalty=0,
        presence_penalty=0,
    )

    response_message = response.choices[0].message
    
    available_functions = {"enrich_entities": enrich_entities}  
    function_name = response_message.tool_calls[0].function.name
    
    function_to_call = available_functions[function_name]
    logging.info(f"function_to_call: {function_to_call}")

    function_args = json.loads(response_message.tool_calls[0].function.arguments)
    logging.info(f"function_args: {function_args}")

    function_response = function_to_call(text, function_args)

    return {"model_response": response, 
            "function_response": function_response}

### 5. Wikipediaリンクでテキストを豊かにしよう

#### 5.1 OpenAIタスクの実行

In [12]:
text = """The Beatles were an English rock band formed in Liverpool in 1960, comprising John Lennon, Paul McCartney, George Harrison, and Ringo Starr."""
result = run_openai_task(labels, text)

 2023-10-20 18:05:51,729 - INFO - function_to_call: <function enrich_entities at 0x0000021D30C462A0>
 2023-10-20 18:05:51,730 - INFO - function_args: {'person': ['John Lennon', 'Paul McCartney', 'George Harrison', 'Ringo Starr'], 'org': ['The Beatles'], 'gpe': ['Liverpool'], 'date': ['1960']}
 2023-10-20 18:06:09,858 - INFO - entity_link_dict: {'John Lennon': 'https://en.wikipedia.org/wiki/John_Lennon', 'Paul McCartney': 'https://en.wikipedia.org/wiki/Paul_McCartney', 'George Harrison': 'https://en.wikipedia.org/wiki/George_Harrison', 'Ringo Starr': 'https://en.wikipedia.org/wiki/Ringo_Starr', 'The Beatles': 'https://en.wikipedia.org/wiki/The_Beatles', 'Liverpool': 'https://en.wikipedia.org/wiki/Liverpool'}


#### 5.2 関数レスポンス

In [13]:
display(Markdown(f"""**Text:** {text}   
                     **Enriched_Text:** {result['function_response']}"""))

**Text:** The Beatles were an English rock band formed in Liverpool in 1960, comprising John Lennon, Paul McCartney, George Harrison, and Ringo Starr.   
                     **Enriched_Text:** [The Beatles](https://en.wikipedia.org/wiki/The_Beatles) were an English rock band formed in [Liverpool](https://en.wikipedia.org/wiki/Liverpool) in 1960, comprising [John Lennon](https://en.wikipedia.org/wiki/John_Lennon), [Paul McCartney](https://en.wikipedia.org/wiki/Paul_McCartney), [George Harrison](https://en.wikipedia.org/wiki/George_Harrison), and [Ringo Starr](https://en.wikipedia.org/wiki/Ringo_Starr).

#### 5.3 トークン使用量

推論コストを見積もるには、レスポンスの"usage"フィールドを解析することができます。モデルごとの詳細なトークンコストは[OpenAI Pricing Guide](https://openai.com/pricing)で確認できます：

In [14]:
# estimate inference cost assuming gpt-3.5-turbo (4K context)
i_tokens  = result["model_response"].usage.prompt_tokens 
o_tokens = result["model_response"].usage.completion_tokens 

i_cost = (i_tokens / 1000) * 0.0015
o_cost = (o_tokens / 1000) * 0.002

print(f"""Token Usage
    Prompt: {i_tokens} tokens
    Completion: {o_tokens} tokens
    Cost estimation: ${round(i_cost + o_cost, 5)}""")

Token Usage
    Prompt: 331 tokens
    Completion: 47 tokens
    Cost estimation: $0.00059
